In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import os
os.chdir('../')
os.environ['CUDA_VISIBLE_DEVICES'] = '2'

import sys
sys.path.append('examples')

import jhutil; jhutil.color_log(1111, )

# Don't Unfold!

In [2]:
import sys
sys.argv = [
    "examples/simple_trainer.py",
    "default",
    "--data_dir",
    "data/360_v2/garden/",
    "--data_factor",
    "4",
    "--port",
    "8082",
    "--result_dir",
    "./results/garden",
]

In [3]:
import json
import math
import os
import time
from dataclasses import dataclass, field
from collections import defaultdict
from typing import Dict, List, Optional, Tuple, Union

import imageio
import nerfview
import numpy as np
import torch
import torch.nn.functional as F
import tqdm
import tyro
import viser
import yaml
from datasets.colmap import Dataset, Parser
from datasets.traj import (
    generate_interpolated_path,
    generate_ellipse_path_z,
    generate_spiral_path,
)
from torch import Tensor
from torch.nn.parallel import DistributedDataParallel as DDP
from torch.utils.tensorboard import SummaryWriter
from torchmetrics.image import PeakSignalNoiseRatio, StructuralSimilarityIndexMeasure
from fused_ssim import fused_ssim
from torchmetrics.image.lpip import LearnedPerceptualImagePatchSimilarity
from typing_extensions import Literal, assert_never
from utils import AppearanceOptModule, CameraOptModule, knn, rgb_to_sh, set_random_seed
from lib_bilagrid import (
    BilateralGrid,
    slice,
    color_correct,
    total_variation_loss,
)

from gsplat.compression import PngCompression
from gsplat.distributed import cli
from gsplat.rendering import rasterization
from gsplat.strategy import DefaultStrategy, MCMCStrategy

In [ ]:

from gsplat.strategy import DefaultStrategy, MCMCStrategy
import tyro
from simple_trainer import Config

configs = {
    "default": (
        "Gaussian splatting training using densification heuristics from the original paper.",
        Config(
            strategy=DefaultStrategy(verbose=True),
        ),
    ),
    "mcmc": (
        "Gaussian splatting training using densification from the paper '3D Gaussian Splatting as Markov Chain Monte Carlo'.",
        Config(
            init_opa=0.5,
            init_scale=0.1,
            opacity_reg=0.01,
            scale_reg=0.01,
            strategy=MCMCStrategy(verbose=True),
        ),
    ),
}
cfg = tyro.extras.overridable_config_cli(configs)
cfg.adjust_steps(cfg.steps_scaler)

# .

In [ ]:
from datasets.colmap import Dataset, Parser

parser = Parser(
    data_dir=cfg.data_dir,
    factor=cfg.data_factor,
    normalize=cfg.normalize_world_space,
    test_every=cfg.test_every,
)

In [ ]:
# extrinsic
# R, tvec, camera_id 가 필요함.
dir(parser.manager.images[1])

In [ ]:
# intrinsic
# fx, fy, cx, cy, camera_type, width, height가 필요함
dir(parser.manager.cameras[1])